# Document-Based Question Answering

### Installation

In [ ]:
!pip install -U langchain-community langchain-chroma pypdf chromadb hf_xet langchain-huggingface huggingface_hub "langchain-google-genai>=0.0.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2 MB/s eta 0:00:00

### Hugging Face Login

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

## Process Document

### Load in document (pdf)

In [ ]:
from langchain.document_loaders import PyPDFLoader
def load(file):
  loader = PyPDFLoader(file)
  pages = loader.load()
  return pages

# TEST
# pages = load('drive/MyDrive/dbqa/PaperQA.pdf')

# print(f"Number of pages: {len(pages)}\n")
# print(f"First 500 characters:\n{pages[0].page_content[0:500]}")
# print(f"\nMeta Data: {pages[0].metadata}")

### Split into chunks
Play around with chunk sizes
- Paper says size of 4000, but might be a little bit too big for prototype?

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split(pages, chunk_size, chunk_overlap):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap
  )

  splits = text_splitter.split_documents(pages)
  return splits

# TEST
# splits = split(pages, 1500, 150)
# print(len(splits))
# print(splits[1])

### Make embedding chunks, create vector database
- Potentially try multiple models to find best result
- Currently all-MiniLM-L6-v2

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

def create_db(splits, persist_directory):
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  vectordb = Chroma.from_documents(
      documents=splits,
      embedding=embedding,
      persist_directory=persist_directory
  )
  return vectordb

Embed Document

In [ ]:
file_name = 'drive/MyDrive/dbqa/PaperQA.pdf'
persist_directory = 'drive/MyDrive/dbqa/paperqa_db'

In [ ]:
pages = load(file_name)
splits = split(pages, 1500, 150)
vectordb = create_db(splits, persist_directory)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Propose Answer

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

def answer(question, vectordb, llm):
  template = """Answer in a direct and concise tone, I am in a hurry. Your audience is an expert, so be
  highly specific. If there are ambiguous terms or acronyms, first define them.
  Write an answer with five sentences maximum for the question below based on the provided context.
  If the context provides insufficient information, reply ''I cannot answer''. Answer in an unbiased, comprehensive,
  and scholarly tone. If the question is subjective, provide an opinionated answer in the concluding 1-2 sentences.

  {context}

  Question: {question}

  Answer:"""

  QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

  retriever = vectordb.as_retriever(
      search_type="mmr",
      search_kwargs={"k": 5, "fetch_k": 10}
  )

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  result = qa_chain.invoke({"query": question})
  return result

## Put it all together

### Setup

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

persist_directory = 'drive/MyDrive/dbqa/paperqa_db'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
# question = "Explain how the search tool works."

###Host on Colab

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from IPython.display import display

llm_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-large",
        max_new_tokens=512,
    )

llm = HuggingFacePipeline(pipeline=llm_pipeline)
result = answer(question, vectordb, llm)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1727 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
display(result['result'])

###Gemini API

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_api_key = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api_key)

question = "What are some sources cited in this paper that I can learn more about retrieval augmented generation?"

result = answer(question, vectordb, llm)
display(result['result'])

'*   **Retrieval-Augmented Generation (RAG)**: Models that reduce hallucinations and provide provenance by retrieving information to generate answers.\n\nThe paper cites several sources relevant to RAG. Borgeaud et al. [67] discuss improving language models by retrieving from trillions of tokens. Lyu et al. [68] focus on improving RAG models via data importance learning. Menick et al. [66] teach language models to support answers with verified quotes. These papers could provide further insights into RAG.'

###Successful Questions
These are questions that the prototype answered correctly about the PaperQA paper.

- "How does the PaperQA model work?"
- "Explain how the search tool works."
- "How did the authors test the performance of the PaperQA model?"
- "How much better did PaperQA perform compared to competing models?" (pulled data from a table)
- "How did the researchers counteract hallucination?"
- "What are some sources cited in this paper that I can learn more about retrieval augmented generation?" (pulled several citations from the paper)
- "What would the cost per hour of PaperQA be?"
- "What experiments were run on PaperQA?
- "What are some examples of questions that were in the LitQA dataset?" (attempted to cite, correctly cited which table it was from but didn't know who the authors were)

###Failed Questions
These are questions that the prototype was unable to answer. Some questions were supposed to fail (denoted) while others had the information contained within the paper.

- "Who are the authors of this paper?"
- "What is the title of this paper?"
- "How do I generate a new Google AI API key?" (failed successfully)
- "If I wanted to build PaperQA myself, how could I do it?" (I didn't expect this to work, but more reasoning based on the findings of the paper could be built in later)
- "What question did I ask you two questions ago?"